#Extracting Frames from video
This "extract_frame_from_video()" function takes your video path and extract frame from your video and save the extracted frame into frames folder

In [ ]:
import cv2
import os

In [ ]:
def extract_frame_from_video(video_file):
  # Create a directory to store the extracted frames
  if not os.path.exists("frames"):
      os.mkdir("frames")

  # Get the video capture object
  cap = cv2.VideoCapture(video_file)

  # Get the frame rate
  fps = cap.get(cv2.CAP_PROP_FPS)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
  print(fps)
  # Iterate over the frames in the video
  for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
      # Capture the frame
      ret, frame = cap.read()
      frame=cv2.resize(frame,(640,480))

      # Save the frame
      filename = os.path.join("frames", "frame_v%d.png" % i)
      cv2.imwrite(filename, frame)

  # Release the video capture object
  cap.release()
  cv2.destroyAllWindows()


In [ ]:
# Set the video file to be extracted
video_file = "videos/d1.mp4" # your video path
extract_frame_from_video(video_file)

#Deleting Duplicate Images

This "remove_duplicates images()" function takes your frames folder path, and use hashing technique to find duplicate images and then delete all the duplicate images from folder

In [ ]:
!pip install imagehash

     ------------------------------------ 296.5/296.5 kB 458.1 kB/s eta 0:00:00


In [ ]:
import os
from PIL import Image
import imagehash

def get_image_hash(image_path):
    # Open the image and calculate its hash value
    with Image.open(image_path) as img:
        img_hash = imagehash.average_hash(img)
    return img_hash

def remove_duplicate_images(folder_path):
    # Check if the provided folder path is valid
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return

    # Get a list of all files (including images) in the folder
    all_files = os.listdir(folder_path)

    # Filter only the image files (you may adjust the valid image extensions)
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # Check if there are any images in the folder
    if not image_files:
        print(f"No image files found in folder '{folder_path}'.")
        return

    # Create a dictionary to store image hashes and their corresponding filenames
    image_hashes = {}

    # List to store duplicate image filenames
    duplicates_to_delete = []

    try:
        # Iterate through all image files in the folder
        for image in image_files:
            image_path = os.path.join(folder_path, image)
            img_hash = get_image_hash(image_path)

            # Check if the hash value already exists in the dictionary
            if img_hash in image_hashes:
                duplicates_to_delete.append(image)  # Add to duplicate list
            else:
                image_hashes[img_hash] = image  # Add to the dictionary

        # Remove duplicate images
        num_deleted_images = 0
        for duplicate_image in duplicates_to_delete:
            duplicate_image_path = os.path.join(folder_path, duplicate_image)
            os.remove(duplicate_image_path)
            num_deleted_images += 1
            print(f"Removed duplicate image: '{duplicate_image}' from folder '{folder_path}'.")

        print(f"Total {num_deleted_images} duplicate images deleted.")

    except Exception as e:
        print(f"Error occurred while removing duplicate images: {e}")


In [ ]:
# put your fraes folder path
folder_path = "D:/renamed"
remove_duplicate_images(folder_path)

# Anonating Images

After frame extraction and remove duplicate images. We need to do annotating the images.

For image annotating I use labelImg tool. It is a open source(free) tool for labeling the image. you an download labelImg from the github repository link :-  https://github.com/HumanSignal/labelImg

1. clone this repo
2. run labelImg.py file in your anaconda prompt
3. label images in PascalVOC format

In [ ]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [ ]:
# Pass your lable image folder path that contains .xml file of image
# Load all xml files from directory
xml_list = glob("D:/Labelled/Data_1/*.xml")

In [ ]:
xml_list

['D:/Labelled/Data_1\\image_0.xml',
 'D:/Labelled/Data_1\\image_1.xml',
 'D:/Labelled/Data_1\\image_100.xml',
 'D:/Labelled/Data_1\\image_103.xml',
 'D:/Labelled/Data_1\\image_104.xml',
 'D:/Labelled/Data_1\\image_105.xml',
 'D:/Labelled/Data_1\\image_106.xml',
 'D:/Labelled/Data_1\\image_108.xml',
 'D:/Labelled/Data_1\\image_11.xml',
 'D:/Labelled/Data_1\\image_113.xml',
 'D:/Labelled/Data_1\\image_114.xml',
 'D:/Labelled/Data_1\\image_116.xml',
 'D:/Labelled/Data_1\\image_118.xml',
 'D:/Labelled/Data_1\\image_119.xml',
 'D:/Labelled/Data_1\\image_12.xml',
 'D:/Labelled/Data_1\\image_123.xml',
 'D:/Labelled/Data_1\\image_124.xml',
 'D:/Labelled/Data_1\\image_128.xml',
 'D:/Labelled/Data_1\\image_13.xml',
 'D:/Labelled/Data_1\\image_133.xml',
 'D:/Labelled/Data_1\\image_134.xml',
 'D:/Labelled/Data_1\\image_135.xml',
 'D:/Labelled/Data_1\\image_136.xml',
 'D:/Labelled/Data_1\\image_137.xml',
 'D:/Labelled/Data_1\\image_138.xml',
 'D:/Labelled/Data_1\\image_139.xml',
 'D:/Labelled/Data_

In [ ]:
# replacing \\ to /

xml_list = list(map(lambda x : x.replace('\\', '/'), xml_list))

In [ ]:
# Reading all xml file
# From each XML file we need extract
# filename, size(width, heigth), object(name, xmin, xmax, ymin, ymax)

def extract_text_from_xml(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract file name
    image_name = root.find('filename').text

    # extract dimension of image
    image_width = root.find('size').find('width').text
    image_height = root.find('size').find('height').text

    parser = []
    # extract name, xmin, xmax, ymin, ymax
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        xmin = obj.find('bndbox').find('xmin').text
        xmax = obj.find('bndbox').find('xmax').text
        ymin = obj.find('bndbox').find('ymin').text
        ymax = obj.find('bndbox').find('ymax').text
        parser.append([image_name, image_width, image_height, obj_name, xmin, xmax, ymin, ymax])

    return parser

In [ ]:
parser = list(map(extract_text_from_xml, xml_list))

In [ ]:
# reducind data dimension 3D to 2D
data = reduce(lambda x, y : x+y, parser)

In [ ]:
data

[['image_0.jpg', '338', '339', 'car', '71', '267', '107', '278'],
 ['image_1.jpg', '670', '600', 'car', '2', '670', '65', '532'],
 ['image_100.jpg', '187', '240', 'car', '21', '127', '141', '236'],
 ['image_103.jpg', '500', '280', 'car', '11', '496', '18', '265'],
 ['image_104.jpg', '320', '224', 'car', '13', '299', '61', '150'],
 ['image_105.jpg', '320', '213', 'car', '26', '313', '81', '198'],
 ['image_106.jpg', '319', '213', 'car', '12', '301', '79', '185'],
 ['image_108.jpg', '360', '240', 'car', '45', '309', '57', '186'],
 ['image_11.jpg', '320', '240', 'car', '9', '155', '86', '188'],
 ['image_11.jpg', '320', '240', 'car', '160', '313', '83', '177'],
 ['image_113.jpg', '320', '214', 'car', '12', '293', '32', '198'],
 ['image_114.jpg', '240', '180', 'car', '49', '189', '19', '152'],
 ['image_116.jpg', '1280', '853', 'car', '251', '932', '262', '615'],
 ['image_118.jpg', '400', '225', 'car', '55', '317', '25', '176'],
 ['image_119.jpg', '319', '213', 'car', '19', '285', '46', '202'

### creating Pandas Dataframe to store above rsults

In [ ]:
df = pd.DataFrame(data, columns = ['image_name', 'image_width', 'image_height', 'obj_name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [ ]:
df

,image_name,image_width,image_height,obj_name,xmin,xmax,ymin,ymax
0,image_0.jpg,338,339,car,71,267,107,278
1,image_1.jpg,670,600,car,2,670,65,532
2,image_100.jpg,187,240,car,21,127,141,236
3,image_103.jpg,500,280,car,11,496,18,265
4,image_104.jpg,320,224,car,13,299,61,150
...,...,...,...,...,...,...,...,...
269,image_90.jpg,872,578,car,62,833,110,539
270,image_91.jpg,2560,1440,car,471,2171,470,1209
271,image_94.jpg,1600,900,car,69,1532,105,870
272,image_97.jpg,319,213,car,29,264,76,203


In [ ]:
df.shape

(274, 8)

In [ ]:
df['image_name'].value_counts()

image_171.jpg    6
image_70.jpg     5
image_12.jpg     4
image_166.jpg    3
image_200.jpg    3
                ..
image_271.jpg    1
image_274.jpg    1
image_284.jpg    1
image_286.jpg    1
image_99.jpg     1
Name: image_name, Length: 251, dtype: int64

In [ ]:
df['obj_name'].value_counts()

car       205
person     69
Name: obj_name, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    274 non-null    object
 1   image_width   274 non-null    object
 2   image_height  274 non-null    object
 3   obj_name      274 non-null    object
 4   xmin          274 non-null    object
 5   xmax          274 non-null    object
 6   ymin          274 non-null    object
 7   ymax          274 non-null    object
dtypes: object(8)
memory usage: 17.2+ KB


In [ ]:
# converting object datatype to int
column = ['image_width', 'image_height', 'xmin', 'xmax', 'ymin', 'ymax']
df[column] = df[column].astype(int)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    274 non-null    object
 1   image_width   274 non-null    int32 
 2   image_height  274 non-null    int32 
 3   obj_name      274 non-null    object
 4   xmin          274 non-null    int32 
 5   xmax          274 non-null    int32 
 6   ymin          274 non-null    int32 
 7   ymax          274 non-null    int32 
dtypes: int32(6), object(2)
memory usage: 10.8+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['obj_name'])

In [ ]:
# calculating center_x, center_y, width and height for yolo format

df['center_x'] = ((df['xmin'] + df['xmax'])/2)/df['image_width']
df['center_y'] = ((df['ymin'] + df['ymax'])/2)/df['image_height']

df['weidth'] = (df['xmax'] - df['xmin'])/df['image_width']
df['height'] = (df['ymax'] - df['ymin'])/df['image_height']


In [ ]:
df.head(60)

,image_name,image_width,image_height,obj_name,xmin,xmax,ymin,ymax,label,center_x,center_y,weidth,height
0,image_0.jpg,338,339,car,71,267,107,278,0,0.500000,0.567847,0.579882,0.504425
1,image_1.jpg,670,600,car,2,670,65,532,0,0.501493,0.497500,0.997015,0.778333
2,image_100.jpg,187,240,car,21,127,141,236,0,0.395722,0.785417,0.566845,0.395833
3,image_103.jpg,500,280,car,11,496,18,265,0,0.507000,0.505357,0.970000,0.882143
4,image_104.jpg,320,224,car,13,299,61,150,0,0.487500,0.470982,0.893750,0.397321
5,image_105.jpg,320,213,car,26,313,81,198,0,0.529687,0.654930,0.896875,0.549296
6,image_106.jpg,319,213,car,12,301,79,185,0,0.490596,0.619718,0.905956,0.497653
7,image_108.jpg,360,240,car,45,309,57,186,0,0.491667,0.506250,0.733333,0.537500
8,image_11.jpg,320,240,car,9,155,86,188,0,0.256250,0.570833,0.456250,0.425000
9,image_11.jpg,320,240,car,160,313,83,177,0,0.739062,0.541667,0.478125,0.391667


In [ ]:
# train test split

images = df['image_name'].unique()

img_df = pd.DataFrame(images, columns=['image_name'])

# select 80% random images
img_train = tuple(img_df.sample(frac = 0.8)['image_name'])

In [ ]:
# train
img_df

,image_name
0,image_0.jpg
1,image_1.jpg
2,image_100.jpg
3,image_103.jpg
4,image_104.jpg
...,...
246,image_90.jpg
247,image_91.jpg
248,image_94.jpg
249,image_97.jpg


In [ ]:
img_test = tuple(img_df.query(f'image_name not in {img_train}')['image_name'])

In [ ]:
# saving training and testing image in folder with there respective txt files

from shutil import move

In [ ]:
train_df = df.query(f'image_name in {img_train}')
test_df = df.query(f'image_name in {img_test}')

In [ ]:
train_folder = 'D:/Labelled/Data_1/data/train'
test_folder = 'D:/Labelled/Data_1/data/test'

# creating folder
os.makedirs(train_folder)
os.makedirs(test_folder)

In [ ]:
column = ['image_name', 'label', 'center_x', 'center_y', 'weidth', 'height']

grouped_train_df = train_df[column].groupby('image_name')

grouped_test_df = test_df[column].groupby('image_name')

In [ ]:
grouped_train_df.head(50)

,image_name,label,center_x,center_y,weidth,height
0,image_0.jpg,1,0.500000,0.567847,0.579882,0.504425
1,image_1.jpg,1,0.501493,0.497500,0.997015,0.778333
2,image_100.jpg,1,0.395722,0.785417,0.566845,0.395833
3,image_103.jpg,1,0.507000,0.505357,0.970000,0.882143
4,image_104.jpg,1,0.487500,0.470982,0.893750,0.397321
...,...,...,...,...,...,...
269,image_90.jpg,1,0.513188,0.561419,0.884174,0.742215
270,image_91.jpg,1,0.516016,0.582986,0.664062,0.513194
271,image_94.jpg,1,0.500313,0.541667,0.914375,0.850000
272,image_97.jpg,1,0.459248,0.654930,0.736677,0.596244


In [ ]:
# grouped_train_df.get_group('image_94.jpg').set_index('image_name').to_csv('sample.txt', index=False, header=False)

In [ ]:
# this function create .txt file for each unique image and save it to destination folder with image

def save_data(image_name, folder_path, grouped_obj):
    src_path = os.path.join('D:/Labelled/Data_1/', image_name)
    dst_path = os.path.join(folder_path, image_name)
    print(src_path," ",dst_path)
    move(src_path, dst_path)

    # saving anotation in .txt file
    text_filename = os.path.join(folder_path , os.path.splitext(image_name)[0] + '.txt')
    grouped_obj.get_group(image_name).set_index('image_name').to_csv(text_filename, sep=' ', index=False, header=False)

In [ ]:
# save_data(image_name, folder_path, grouped_obj)
image_name = pd.Series(grouped_train_df.groups.keys())

In [ ]:
image_name.apply(save_data, args = (train_folder, grouped_train_df))

D:/Labelled/Data_1/image_0.jpg   D:/Labelled/Data_1/data/train\image_0.jpg
D:/Labelled/Data_1/image_1.jpg   D:/Labelled/Data_1/data/train\image_1.jpg
D:/Labelled/Data_1/image_100.jpg   D:/Labelled/Data_1/data/train\image_100.jpg
D:/Labelled/Data_1/image_103.jpg   D:/Labelled/Data_1/data/train\image_103.jpg
D:/Labelled/Data_1/image_104.jpg   D:/Labelled/Data_1/data/train\image_104.jpg
D:/Labelled/Data_1/image_106.jpg   D:/Labelled/Data_1/data/train\image_106.jpg
D:/Labelled/Data_1/image_108.jpg   D:/Labelled/Data_1/data/train\image_108.jpg
D:/Labelled/Data_1/image_11.jpg   D:/Labelled/Data_1/data/train\image_11.jpg
D:/Labelled/Data_1/image_113.jpg   D:/Labelled/Data_1/data/train\image_113.jpg
D:/Labelled/Data_1/image_114.jpg   D:/Labelled/Data_1/data/train\image_114.jpg
D:/Labelled/Data_1/image_116.jpg   D:/Labelled/Data_1/data/train\image_116.jpg
D:/Labelled/Data_1/image_119.jpg   D:/Labelled/Data_1/data/train\image_119.jpg
D:/Labelled/Data_1/image_12.jpg   D:/Labelled/Data_1/data/trai

D:/Labelled/Data_1/image_726.jpg   D:/Labelled/Data_1/data/train\image_726.jpg
D:/Labelled/Data_1/image_728.jpg   D:/Labelled/Data_1/data/train\image_728.jpg
D:/Labelled/Data_1/image_729.jpg   D:/Labelled/Data_1/data/train\image_729.jpg
D:/Labelled/Data_1/image_73.jpg   D:/Labelled/Data_1/data/train\image_73.jpg
D:/Labelled/Data_1/image_731.jpg   D:/Labelled/Data_1/data/train\image_731.jpg
D:/Labelled/Data_1/image_732.jpg   D:/Labelled/Data_1/data/train\image_732.jpg
D:/Labelled/Data_1/image_734.jpg   D:/Labelled/Data_1/data/train\image_734.jpg
D:/Labelled/Data_1/image_736.jpg   D:/Labelled/Data_1/data/train\image_736.jpg
D:/Labelled/Data_1/image_741.jpg   D:/Labelled/Data_1/data/train\image_741.jpg
D:/Labelled/Data_1/image_744.jpg   D:/Labelled/Data_1/data/train\image_744.jpg
D:/Labelled/Data_1/image_745.jpg   D:/Labelled/Data_1/data/train\image_745.jpg
D:/Labelled/Data_1/image_749.jpg   D:/Labelled/Data_1/data/train\image_749.jpg
D:/Labelled/Data_1/image_752.jpg   D:/Labelled/Data_1/

0      None
1      None
2      None
3      None
4      None
       ... 
196    None
197    None
198    None
199    None
200    None
Length: 201, dtype: object

In [ ]:
image_name = pd.Series(grouped_test_df.groups.keys())
image_name.apply(save_data, args = (test_folder, grouped_test_df))

D:/Labelled/Data_1/image_105.jpg   D:/Labelled/Data_1/data/test\image_105.jpg
D:/Labelled/Data_1/image_118.jpg   D:/Labelled/Data_1/data/test\image_118.jpg
D:/Labelled/Data_1/image_145.jpg   D:/Labelled/Data_1/data/test\image_145.jpg
D:/Labelled/Data_1/image_15.jpg   D:/Labelled/Data_1/data/test\image_15.jpg
D:/Labelled/Data_1/image_151.jpg   D:/Labelled/Data_1/data/test\image_151.jpg
D:/Labelled/Data_1/image_158.jpg   D:/Labelled/Data_1/data/test\image_158.jpg
D:/Labelled/Data_1/image_164.jpg   D:/Labelled/Data_1/data/test\image_164.jpg
D:/Labelled/Data_1/image_186.jpg   D:/Labelled/Data_1/data/test\image_186.jpg
D:/Labelled/Data_1/image_188.jpg   D:/Labelled/Data_1/data/test\image_188.jpg
D:/Labelled/Data_1/image_19.jpg   D:/Labelled/Data_1/data/test\image_19.jpg
D:/Labelled/Data_1/image_201.jpg   D:/Labelled/Data_1/data/test\image_201.jpg
D:/Labelled/Data_1/image_202.jpg   D:/Labelled/Data_1/data/test\image_202.jpg
D:/Labelled/Data_1/image_230.jpg   D:/Labelled/Data_1/data/test\imag

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
dtype: object